<a href="https://colab.research.google.com/github/ishancoderr/Neural-networks/blob/main/Boltzman_machine.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install -q torch==1.0.0 torchvision


     |████████████████████████████████| 591.8MB 27kB/s 
ERROR: torchvision 0.8.1+cu101 has requirement torch==1.7.0, but you'll have torch 1.0.0 which is incompatible.


In [11]:
import torch
print(torch.__version__)

1.0.0


In [12]:
!pip install torch

In [46]:
import numpy as np
import pandas as pd


In [47]:
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.optim as optim
import torch.utils.data
from torch.autograd import Variable


In [48]:
url="https://raw.githubusercontent.com/ishancoderr/Neural-networks/main/movies.dat"
url2="https://raw.githubusercontent.com/ishancoderr/Neural-networks/main/users.dat"
url3="https://raw.githubusercontent.com/ishancoderr/Neural-networks/main/ratings.dat"
movies=pd.read_csv(url,sep='::',header=None,engine='python',encoding='latin-1')
users=pd.read_csv(url2,sep='::',header=None,engine='python',encoding='latin-1')
ratings=pd.read_csv(url3,sep='::',header=None,engine='python',encoding='latin-1')

In [49]:
url4="https://raw.githubusercontent.com/ishancoderr/Neural-networks/main/u1.base"
training_set =pd.read_csv(url4,delimiter='\t')
training_set=np.array(training_set,dtype='int')
test_set=pd.read_csv(url4,delimiter='\t')
test_set=np.array(test_set,dtype='int')

**Getting the number of users and movies**

In [50]:
nb_users = int(max(max(training_set[:, 0], ), max(test_set[:, 0])))
nb_movies = int(max(max(training_set[:, 1], ), max(test_set[:, 1])))

In [51]:
def convert(data):
  new_data = []
  for id_users in range(1, nb_users + 1):
    id_movies = data[:, 1] [data[:, 0] == id_users]
    id_ratings = data[:, 2] [data[:, 0] == id_users]
    ratings = np.zeros(nb_movies)
    ratings[id_movies - 1] = id_ratings
    new_data.append(list(ratings))
  return new_data
training_set = convert(training_set)
test_set = convert(test_set)



In [52]:
training_set = torch.FloatTensor(training_set)
test_set = torch.FloatTensor(test_set)

In [53]:
training_set[training_set == 0] = -1
training_set[training_set == 1] = 0
training_set[training_set == 2] = 0
training_set[training_set >= 3] = 1
test_set[test_set == 0] = -1
test_set[test_set == 1] = 0
test_set[test_set == 2] = 0
test_set[test_set >= 3] = 1

In [54]:
class RBM():
  def __init__(self, nv, nh):
    self.W = torch.randn(nh, nv)
    self.a = torch.randn(1, nh)
    self.b = torch.randn(1, nv)
  def sample_h(self, x):
    wx = torch.mm(x, self.W.t())
    activation = wx + self.a.expand_as(wx)
    p_h_given_v = torch.sigmoid(activation)
    return p_h_given_v, torch.bernoulli(p_h_given_v)
  def sample_v(self, y):
    wy = torch.mm(y, self.W)
    activation = wy + self.b.expand_as(wy)
    p_v_given_h = torch.sigmoid(activation)
    return p_v_given_h, torch.bernoulli(p_v_given_h)
  def train(self, v0, vk, ph0, phk):
    self.W += (torch.mm(v0.t(), ph0) - torch.mm(vk.t(), phk)).t()
    self.b += torch.sum((v0 - vk), 0)
    self.a += torch.sum((ph0 - phk), 0)
nv = len(training_set[0])
nh = 100
batch_size = 100
rbm = RBM(nv, nh)

**Training the RBM**

In [ ]:
nb_epoch=10
for epoch in range(1,nb_epoch+1):
  train_loss =0
  s=0.
  for id_user in range(0,nb_users-batch_size):
    vk = training_set[id_user : id_user + batch_size]
    v0 = training_set[id_user : id_user + batch_size]
    ph0,_ = rbm.sample_h(v0)
    for k in range(10):
      _,hk=rbm.sample_h(vk)
      _,vk=rbm.sample_v(hk)
      vk[v0<0] = v0[v0<0]
    phk,_ = rbm.sample_h(vk)
    rbm.train(v0, vk, ph0, phk)
    train_loss += torch.mean(torch.abs(v0[v0 >= 0] - vk[v0 >= 0]))
    s += 1.
  print('epoch: '+str(epoch)+' loss: '+str(train_loss/s))
